# Graph Neural Networks
## What are Graph Neural Networks (GNNs)?

In [1]:
#import the basics
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch_geometric
import torch.nn as nn
from torch import optim, Tensor
import torch.nn.functional as F
import scipy.sparse as sp
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip, HeteroData
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.datasets import AmazonBook, MovieLens
from torch_geometric.transforms import Compose, ToDevice, ToUndirected, RandomLinkSplit
from torch_geometric.data import Data
from torch_geometric.typing import Adj
from torch_sparse import SparseTensor, matmul
from torch_geometric.utils import train_test_split_edges
%matplotlib inline

In [2]:
torch_geometric.seed_everything(1234)
torch_geometric.__version__

'2.6.1'

In [3]:
# Let's verify what device we are working with
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


Graph Neural Networks are a type of "geometric deep learning" models that use pairwise message passing. They typically have an architecture consisting of 3 types of layers. From [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network):
1. Permutation equivariant: a permutation equivariant layer maps a representation of a graph into an updated representation of the same graph. In the literature, permutation equivariant layers are implemented via **pairwise message passing between graph nodes**. Intuitively, in a message passing layer, nodes update their representations by aggregating the messages received from their immediate neighbours. As such, each message passing layer increases the receptive field of the GNN by one hop.
2. Local pooling: a local pooling layer coarsens the graph via downsampling. Local pooling is used to increase the receptive field of a GNN, in a similar fashion to pooling layers in convolutional neural networks. Examples include k-nearest neighbours pooling, top-k pooling, and self-attention pooling.
3. Global pooling: a global pooling layer, also known as readout layer, provides fixed-size representation of the whole graph. The global pooling layer must be permutation invariant, such that permutations in the ordering of graph nodes and edges do not alter the final output. Examples include element-wise sum, mean or maximum.

## Attributes
- [T]he preprocessing step first
“squashes” the graph structured data into a vector of reals and
then deals with the preprocessed data using a list-based data
processing technique. However, important information, e.g., the
topological dependency of information on each node may be
lost during the preprocessing stage and the final result may depend, in an unpredictable manner, on the details of the preprocessing algorith [1] **GNNS preserve the structure of the graph it is based on.**
- It will be shown that the GNN
is an extension of both recursive neural networks and random
walk models and that it retains their characteristics. The model
extends recursive neural networks since it can process a more
general class of graphs including cyclic, directed, and undirected graphs, and it can deal with node-focused applications
without any preprocessing steps. The approach extends random
walk theory by the introduction of a learning algorithm and by
enlarging the class of processes that can be modeled. [1]
- Weights are shared across layer structures

### What is message passing?
From [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network#Message_passing_layers):
<br>
![img](./img/notebook/messagePassing.png)

## Computation Graph
"The neighbour of a node defines its computation graph" - @12:34 https://www.youtube.com/watch?v=JtDgmmQ60x8&ab_channel=AntonioLonga



## Data

In [55]:
transform = Compose([ToDevice(device)])
movielens_dataset = MovieLens(root="./data/MovieLens", transform=transform, model_name='all-MiniLM-L6-v2')
movielens_dataset.process()
print(f"Dataset: {movielens_dataset}")
print(f"Number of graphs in dataset: {len(movielens_dataset)}")
print(f"Number of features of dataset: {movielens_dataset.num_features}")

Batches:   0%|          | 0/305 [00:00<?, ?it/s]

C:\Users\Rob\miniconda3\envs\gnn-ass-env\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Dataset: MovieLens()
Number of graphs in dataset: 1
Number of features of dataset: {'movie': 404, 'user': 0}


In [56]:
movielens_dataset[0]

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836],
    time=[100836],
  }
)

In [57]:
movielens_dataset[0]['user','rates','movie']['edge_index'].cpu().numpy()[:,:5]

array([[ 0,  0,  0,  0,  0],
       [ 0,  2,  5, 43, 46]], dtype=int64)

In [76]:
torch.where(movielens_dataset[0]['user','rates','movie']['edge_label'] >= 4, movielens_dataset[0]['user','rates','movie']['edge_label'], 0)

tensor([4, 4, 4,  ..., 5, 5, 0], device='cuda:0')

In [73]:
from torch_geometric.data import HeteroData
data = HeteroData()

data["user"].node_id = torch.arange(movielens_dataset[0]['user']['num_nodes'])
data["movie"].node_id = torch.arange(len(movielens_dataset[0]['user','rates','movie']['edge_index'][1].unique()))
data['user','rates','movie'].edge_index = movielens_dataset[0]['user','rates','movie']['edge_index']

In [74]:
data

HeteroData(
  user={ node_id=[610] },
  movie={ node_id=[9724] },
  (user, rates, movie)={ edge_index=[2, 100836] }
)

In [83]:
import torch_geometric.transforms as T
transform = T.Compose([T.RandomLinkSplit(
    is_undirected=False,
    edge_types=('user','rates','movie'),
    split_labels=True,
), ToDevice(device)])

train_data, val_data, test_data = transform(data)

In [84]:
train_data

HeteroData(
  user={ node_id=[610] },
  movie={ node_id=[9724] },
  (user, rates, movie)={
    edge_index=[2, 70586],
    pos_edge_label=[70586],
    pos_edge_label_index=[2, 70586],
    neg_edge_label=[70586],
    neg_edge_label_index=[2, 70586],
  }
)

In [85]:
val_data

HeteroData(
  user={ node_id=[610] },
  movie={ node_id=[9724] },
  (user, rates, movie)={
    edge_index=[2, 70586],
    pos_edge_label=[10083],
    pos_edge_label_index=[2, 10083],
    neg_edge_label=[10083],
    neg_edge_label_index=[2, 10083],
  }
)

In [87]:
test_data

HeteroData(
  user={ node_id=[610] },
  movie={ node_id=[9724] },
  (user, rates, movie)={
    edge_index=[2, 80669],
    pos_edge_label=[20167],
    pos_edge_label_index=[2, 20167],
    neg_edge_label=[20167],
    neg_edge_label_index=[2, 20167],
  }
)

In [106]:
print(train_data['user','rates','movie']['pos_edge_label_index'][:,:5])
print(train_data['user','rates','movie']['neg_edge_label_index'][:,:5])
print(train_data['user','rates','movie']['edge_index'][:,:5])

tensor([[ 425,   19,  560,  606,  437],
        [1777, 1593, 6770,  701, 5819]], device='cuda:0')
tensor([[ 593,  536,  347,  409,  241],
        [ 458, 5854, 5834, 9720, 6206]], device='cuda:0')
tensor([[ 425,   19,  560,  606,  437],
        [1777, 1593, 6770,  701, 5819]], device='cuda:0')
tensor([[ 306,  225,  560,  ...,  211,  390,  209],
        [1445, 6481, 3363,  ..., 7465, 1286, 5023]], device='cuda:0')


## Neural Graph Collaborative Filtering



In [7]:
from torch_geometric.nn import Linear
from torch.nn import Embedding
from torch.nn import functional as F

class EmbeddingPropLayer(torch.nn.Module):
    def __init__(self, hidden_channels=128):
        super(EmbeddingPropLayer, self).__init__()

        self.W1 = Linear(hidden_channels, hidden_channels, bias=False)
        self.W2 = Linear(hidden_channels, hidden_channels, bias=False)

    def reset_parameters(self):
        self.W1.reset_parameters()
        self.W2.reset_parameters()

    def forward(self, E, E_final):
        message = self.message_aggregation(E)
        return message, torch.concat([E_final, message], dim=1)

    def message_construction(self, E, p_ui):
        return torch.mul(p_ui*E, p_ui*self.W2(E))

    def message_aggregation(self, E):
        p_ui = 1
        m_ui = self.message_construction(E, p_ui)
        m_uu = p_ui*self.W1(E)
        return F.leaky_relu(m_uu + m_ui)


class EmbeddingLayer(torch.nn.Module):
    """
        user u and item i with an embedding vector e_u that's an element of d real numbers
        and e_i that's an element of a d real numbers
    """
    def __init__(self, num_users, num_movies, hidden_channels=128):
        super(EmbeddingLayer, self).__init__()
        self.user_embedding = Embedding(num_users, hidden_channels)
        self.movie_embedding = Embedding(num_movies, hidden_channels)

    def forward(self, user_nodes, movie_nodes):
        e_u = self.user_embedding(user_nodes)
        e_i = self.movie_embedding(movie_nodes)
        E = torch.concat([e_u, e_i])

        return E

class NGCF(torch.nn.Module):
    def __init__(self, num_of_edges, hidden_channels=128):
        super(NGCF, self).__init__()
        self.num_of_edges = num_of_edges
        self.hidden_channels = hidden_channels

        self.embedding_layer = Embedding(self.num_of_edges, hidden_channels)

        self.embedding_prop_layer_1 = EmbeddingPropLayer(hidden_channels)
        self.embedding_prop_layer_2 = EmbeddingPropLayer(hidden_channels)
        self.embedding_prop_layer_3 = EmbeddingPropLayer(hidden_channels)


    def forward(self, edge_index, num_users):
        E = self.embedding_layer(edge_index) # size -> [num_users + num_movies, hidden_channels]

        #assert E.size()[0] == self.num_of_edges and E.size()[1] == self.hidden_channels

        E_1, E_star = self.embedding_prop_layer_1(E, torch.empty_like(E)) #E_l -> [num_users+num_movies,
        E_2, E_star = self.embedding_prop_layer_2(E_1, E_star)
        E_3, E_star = self.embedding_prop_layer_2(E_2, E_star)

        #assert E_star.size()[0] == self.num_of_edges and E_star.size()[1] == self.hidden_channels*4

        e_u_star = E_star[:(num_users-1), :]
        e_i_star = E_star[num_users:, :]

        print(f"self number of users: {self.num_of_users}")
        print(f"number of users: {num_users}")
        print(f"E size {E.size()}")
        print(f"E star size {E_star.size()}")
        print(f"e_u star size {e_u_star.size()}")
        print(f"e_i star size {e_i_star.size()}")
        print(f"e_u 0 size {E[:(num_users-1), :].size()}")
        print(f"e_i 0 size {E_star[num_users:, :].size()}")

        # users_emb_final, users_emb_0, items_emb_final, items_emb_0
        return e_u_star, E[:(num_users-1), :], e_i_star, E[num_users:, :]


In [8]:
class MPNGCF(torch_geometric.nn.MessagePassing):
    def __init__(self, num_of_users, num_of_movies, K=3, add_self_loops=False):
        super(MPNGCF, self).__init__()
        self.num_of_users = num_of_users
        self.num_of_movies = num_of_movies
        self.num_of_edges = num_of_users + num_of_movies

        self.K=K
        self.add_self_loops = add_self_loops

        self.W1 = Linear(self.num_of_users, self.num_of_users, bias=False)
        self.W2 = Linear(self.num_of_users, self.num_of_users, bias=False)

    def forward(self, edge_index):
        edge_index_norm = gcn_norm(
            edge_index, add_self_loops=self.add_self_loops)

        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight]) # E^0
        embs = [emb_0]
        emb_k = emb_0

        print(f"edge_index_norm size: {edge_index_norm.size()}")

        # multi-scale diffusion
        for i in range(self.K):
            emb_k = self.propagate(edge_index_norm, x=emb_k)
            embs.append(emb_k)

        emb_final = torch.stack(embs, dim=1)

        users_emb_final, items_emb_final = torch.split(
            emb_final, [self.num_users, self.num_items]) # splits into e_u^K and e_i^K

        # returns e_u^K, e_u^0, e_i^K, e_i^0
        return users_emb_final, self.users_emb.weight, items_emb_final, self.items_emb.weight

    def message_and_aggregate(self, adj_t):
        print(f"message_and_aggregate adjacency map: {adj_t.size()}")
        m_ui = torch.mul(adj_t, self.W2(adj_t))
        m_uu = self.W1(adj_t)
        return F.leaky_relu(m_uu + m_ui)


In [9]:
def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val):
    """Bayesian Personalized Ranking Loss as described in https://arxiv.org/abs/1205.2618
    Args:
        users_emb_final (torch.Tensor): e_u_k
        users_emb_0 (torch.Tensor): e_u_0
        pos_items_emb_final (torch.Tensor): positive e_i_k
        pos_items_emb_0 (torch.Tensor): positive e_i_0
        neg_items_emb_final (torch.Tensor): negative e_i_k
        neg_items_emb_0 (torch.Tensor): negative e_i_0
        lambda_val (float): lambda value for regularization loss term

    Returns:
        torch.Tensor: scalar bpr loss value
    """
    reg_loss = lambda_val * (users_emb_0.norm(2).pow(2) +
                             pos_items_emb_0.norm(2).pow(2) +
                             neg_items_emb_0.norm(2).pow(2))

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final)
    pos_scores = torch.sum(pos_scores, dim=-1)
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final)
    neg_scores = torch.sum(neg_scores, dim=-1)
    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss

    return loss

In [10]:
def sample_batch(batch_size, edge_index):
 # returns a tuple of 3 tensors. Tensor 1 -> user, Tensor 2 -> positive interactions, Tensor3-> Neg interactions
   edges = structured_negative_sampling(edge_index)
   edges = torch.stack(edges, dim=0)
   indices = random.choices(
        [i for i in range(edges[0].shape[0])], k=batch_size)
   batch = edges[:, indices]
   user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
   return user_indices, pos_item_indices, neg_item_indices


In [11]:
def get_ground_truth(edge_index):
    """Generates dictionary of positive items for each user efficiently.
    Args:
        edge_index (torch.Tensor): 2 by N list of edges
    Returns:
        dict: dictionary of positive items for each user
    """
    user_pos_items = {user.item(): [] for user in edge_index[0].unique()}
    for user, item in zip(edge_index[0], edge_index[1]):
        user_pos_items[user.item()].append(item.item())

    return user_pos_items

In [12]:
def metrics(groundTruth, r, k):
    num_correct_pred = torch.sum(r, dim=-1).float()
    user_num_liked = torch.tensor([len(groundTruth[i]) for i in range(len(groundTruth))], dtype=torch.float)
    recall = torch.mean(num_correct_pred / user_num_liked)
    precision = torch.mean(num_correct_pred) / k
    return recall.item(), precision.item()

In [13]:
# wrapper function to get evaluation metrics
def get_metrics(model, edge_index, exclude_edge_indices, k):
    """Computes the evaluation metrics: recall, precision, and ndcg @ k

    Args:
        model (LighGCN): lightgcn model
        edge_index (torch.Tensor): 2 by N list of edges for split to evaluate
        exclude_edge_indices ([type]): 2 by N list of edges for split to discount from evaluation
        k (int): determines the top k items to compute metrics on

    Returns:
        tuple: recall @ k, precision @ k, ndcg @ k
    """
    user_embedding = model.users_emb.weight
    item_embedding = model.items_emb.weight

    rating = torch.matmul(user_embedding, item_embedding.T)

    for exclude_edge_index in exclude_edge_indices:
        user_pos_items = get_ground_truth(exclude_edge_index)
        exclude_users = []
        exclude_items = []
        for user, items in user_pos_items.items():
            exclude_users.extend([user] * len(items))
            exclude_items.extend(items)

        rating[exclude_users, exclude_items] = -(1 << 10)

    _, top_K_items = torch.topk(rating, k=k)

    users = edge_index[0].unique()

    test_user_pos_items = get_ground_truth(edge_index)

    test_user_pos_items_list = [
        test_user_pos_items[user.item()] for user in users]

    r = []
    for user in users:
        ground_truth_items = test_user_pos_items[user.item()]
        label = list(map(lambda x: x in ground_truth_items, top_K_items[user]))
        r.append(label)
    r = torch.Tensor(np.array(r).astype('float'))

    recall, precision = metrics(test_user_pos_items_list, r, k)
    # ndcg =

    return recall, precision, 0

In [14]:
def evaluation(model, edge_index, sparse_edge_index, exclude_edge_indices, k, lambda_val):
    """Evaluates model loss and metrics including recall, precision, ndcg @ k

    Args:
        model (LighGCN): lightgcn model
        edge_index (torch.Tensor): 2 by N list of edges for split to evaluate
        sparse_edge_index (sparseTensor): sparse adjacency matrix for split to evaluate
        exclude_edge_indices ([type]): 2 by N list of edges for split to discount from evaluation
        k (int): determines the top k items to compute metrics on
        lambda_val (float): determines lambda for bpr loss

    Returns:
        tuple: bpr loss, recall @ k, precision @ k, ndcg @ k
    """

    print(f"sparce edge index {sparse_edge_index}")
    print(f"sparce edge index {sparse_edge_index.storage.row().size()}")
    print(f"sparce edge index {sparse_edge_index.storage.col().size()}")

    edge_index_dense = sparse_edge_index.to_dense().to(torch.long)

    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
        edge_index_dense, len(edge_index_dense))
    edges = structured_negative_sampling(
        edge_index, contains_neg_self_loops=False)
    user_indices, pos_item_indices, neg_item_indices = edges[0], edges[1], edges[2]
    users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]

    print(f"edge index size {edge_index.size()}")
    print(f"movie final size {items_emb_final.shape}")
    print(f"pos item indices size {pos_item_indices.shape}")

    pos_items_emb_final, pos_items_emb_0 = items_emb_final[
        pos_item_indices], items_emb_0[pos_item_indices]
    neg_items_emb_final, neg_items_emb_0 = items_emb_final[
        neg_item_indices], items_emb_0[neg_item_indices]

    loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0,
                    neg_items_emb_final, neg_items_emb_0, lambda_val).item()

    recall, precision, ndcg = get_metrics(
        model, edge_index, exclude_edge_indices, k)

    return loss, recall, precision, ndcg

In [15]:
ITERATIONS = 10000
BATCH_SIZE = 1024
LR = 1e-3
ITERS_PER_EVAL = 200
ITERS_PER_LR_DECAY = 200
K = 20
LAMBDA = 1e-6

In [16]:
model = NGCF(len(2*train_sparse.size()[0]), hidden_channels=128)
#model = MPNGCF(len(train_sparse.storage.row()), len(train_sparse.storage.col()))
model.train()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

print(model)

NGCF(
  (embedding_layer): Embedding(77728, 128)
  (embedding_prop_layer_1): EmbeddingPropLayer(
    (W1): Linear(128, 128, bias=False)
    (W2): Linear(128, 128, bias=False)
  )
  (embedding_prop_layer_2): EmbeddingPropLayer(
    (W1): Linear(128, 128, bias=False)
    (W2): Linear(128, 128, bias=False)
  )
  (embedding_prop_layer_3): EmbeddingPropLayer(
    (W1): Linear(128, 128, bias=False)
    (W2): Linear(128, 128, bias=False)
  )
)


In [28]:
print(train_sparse.storage.row().size())

torch.Size([38864])


In [19]:
train_losses = []
val_losses = []

from tqdm import tqdm

for iter in tqdm(range(ITERATIONS)):
   print(f"row size {train_sparse.storage.row().size()}")
   train_dense = train_sparse.to_dense().to(torch.long)
   users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
       train_dense, len(train_dense))

   user_indices, pos_item_indices, neg_item_indices = sample_batch(
       BATCH_SIZE, train_edge_index)
   user_indices, pos_item_indices, neg_item_indices = user_indices.to(
       device), pos_item_indices.to(device), neg_item_indices.to(device)
   users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
   pos_items_emb_final, pos_items_emb_0 = items_emb_final[
       pos_item_indices], items_emb_0[pos_item_indices]
   neg_items_emb_final, neg_items_emb_0 = items_emb_final[
       neg_item_indices], items_emb_0[neg_item_indices]

   train_loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final,
                         pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, LAMBDA)

   optimizer.zero_grad()
   train_loss.backward()
   optimizer.step()

   if iter % ITERS_PER_EVAL == 0:
       model.eval()
       val_loss, recall, precision, ndcg = evaluation(
           model, val_edge_index, val_sparse, [train_edge_index], K, LAMBDA)
       print(f"[Iteration {iter}/{ITERATIONS}] train_loss: {round(train_loss.item(), 5)}, val_loss: {round(val_loss, 5)}, val_recall@{K}: {round(recall, 5)}, val_precision@{K}: {round(precision, 5)}, val_ndcg@{K}: {round(ndcg, 5)}")
       train_losses.append(train_loss.item())
       val_losses.append(val_loss)
       model.train()

   if iter % ITERS_PER_LR_DECAY == 0 and iter != 0:
       scheduler.step()

  0%|          | 0/10000 [00:00<?, ?it/s]

row size torch.Size([38864])


  0%|          | 0/10000 [00:03<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 51.10 GiB. GPU 0 has a total capacity of 8.00 GiB of which 5.27 GiB is free. Of the allocated memory 1.64 GiB is allocated by PyTorch, and 296.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)